## ChatGPT with LangChain

This notebook provides a quick introduction to ChatGPT and related features supported in LangChain.

Install these libraries before getting started. Ideally, you want to create a dedicated environment for this.

In [4]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

We are adapting code from [here](https://langchain.readthedocs.io/en/latest/modules/chat/getting_started.html).

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [7]:
# chat mode instance
chat = ChatOpenAI(temperature=0)

C:\Users\ROG\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


ChatGPT support different types of messages identifiable by the role. LangChain. Recall how we make a basic call to ChatGPT using `openai`? Here is an example:

```python
MODEL = "gpt-3.5-turbo"

response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are an AI research assistant. You use a tone that is technical and scientific."},
        {"role": "user", "content": "Hello, who are you?"},
        {"role": "assistant", "content": "Greeting! I am an AI research assistant. How can I help you today?"},
        {"role": "user", "content": "Can you tell me about the creation of black holes?"}
    ],
    temperature=0,
)
```

LangChain supports these different types of messages, including a arbitrary role parameter (`ChatMessage`). Let's try: 

In [8]:
USER_INPUT = "I love programming."
FINAL_PROMPT = """Classify the text into neutral, negative or positive. 

Text: {user_input}. 
Sentiment:"""

chat.invoke([HumanMessage(content=FINAL_PROMPT.format(user_input=USER_INPUT))])

AIMessage(content='Positive', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 29, 'total_tokens': 30}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-03056789-baf5-4014-a7e1-aab465e7b21b-0')

Let's try an example that involves a system instruction and a task provided by user.

In [9]:
messages = [
    SystemMessage(content="You are a helpful assistant that can classify the sentiment of input texts. The labels you can use are positive, negative and neutral."),
    HumanMessage(content="Classify the following sentence: I am doing brilliant today!"),
]

chat.invoke(messages)

AIMessage(content='positive', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 49, 'total_tokens': 50}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3d715707-561a-4b70-84d2-a2e40f31576d-0')

Now let's try another example that involves an exchange between a human and AI research assistant:

In [10]:
messages = [
    SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
    HumanMessage(content="Hello, who are you?"),
    AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
    HumanMessage(content="Can you tell me about the creation of black holes?")
]

chat.invoke(messages)

AIMessage(content='Black holes are formed when massive stars exhaust their nuclear fuel and undergo a supernova explosion. If the core of the star is more than about three times the mass of the Sun, it collapses under its own gravity, forming a singularity—a point of infinite density and zero volume. This singularity is surrounded by an event horizon, beyond which nothing, not even light, can escape due to the immense gravitational pull. This creates a region of spacetime from which no information or matter can escape, leading to the formation of a black hole.', response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 70, 'total_tokens': 178}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ddc45ead-9c44-4969-9c54-3d04ccd75abb-0')

There is even a feature to batch these requests and generate response (using `chat.response()`) like so:

In [11]:
batch_messages = [
    [
        SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
        HumanMessage(content="Hello, who are you?"),
        AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
        HumanMessage(content="Can you tell me about the creation of black holes?")
    ],
    [
        SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
        HumanMessage(content="Hello, who are you?"),
        AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
        HumanMessage(content="Can you explain the dark matter?")
    ]
]

chat.generate(batch_messages)

LLMResult(generations=[[ChatGeneration(text='Black holes are formed when massive stars exhaust their nuclear fuel and undergo a supernova explosion. If the core of the star is more than about three times the mass of the Sun, it collapses under its own gravity, forming a black hole. This collapse creates a region of spacetime with a gravitational field so intense that nothing, not even light, can escape from it. This point of infinite density at the center of a black hole is called a singularity. The boundary surrounding the singularity is known as the event horizon, beyond which nothing can escape the gravitational pull of the black hole.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='Black holes are formed when massive stars exhaust their nuclear fuel and undergo a supernova explosion. If the core of the star is more than about three times the mass of the Sun, it collapses under its own gravity, forming a black hole. This collapse creates a r

If you look at the examples above it might be easier to just use a prompt template. LangChain also supports. Let's try that below:

In [12]:
template = "You are a helpful assistant that can classify the sentiment of input texts. The labels you can use are {sentiment_labels}. Classify the following sentence:"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [13]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


chat.invoke(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="I am doing brilliant today!").to_messages())

AIMessage(content='positive', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 50, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0de6d689-b940-4abb-b2f1-4c5a0c970269-0')

In [14]:
chat.invoke(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="Not sure what the weather is like today.").to_messages())

AIMessage(content='Neutral', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 53, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ea9481ae-8d87-448a-ad9e-9e678e0ee358-0')